# **モデル**

In [1]:
%matplotlib inline
import torch
import torch.nn as nn

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import codecs
import os
import re
from tqdm import tqdm

# CWT
import math
import pycwt

# 自作関数
import sys
sys.path.append('..')
import sig_proc
peak_detect = __import__('2_peak_detect') # 先頭が数字のファイルなので素直にimportできない　非推奨だがこのやり方ならOK

def zscore(x,axis=None):
    x_mean = x.mean(axis=axis, keepdims=True)
    x_std = np.std(x, axis=axis, keepdims=True)
    z_score = (x-x_mean)/x_std
    return z_score
    
print(torch.cuda.is_available())

True


In [2]:
import math
# CONV2D->https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html
def Conv2d_OutputShape(in_size,kernel_size,stride=(1,1),padding=(0,0),dilation=(1,1)):
    H_in = in_size[0]
    W_in = in_size[1]
    H_out = math.floor((H_in+2*padding[0]-dilation[0]*(kernel_size[0]-1)-1)/stride[0]+1)
    W_out = math.floor((W_in+2*padding[1]-dilation[1]*(kernel_size[1]-1)-1)/stride[1]+1)
    return (H_out,W_out)

# MAXPOOL2D (CONV2Dと同じ式でした......)->https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html
def MaxPool2d_OutputShape(in_size,kernel_size,stride=None,padding=(0,0),dilation=(1,1)):
    if stride == None:
        stride = kernel_size
    H_in = in_size[0]
    W_in = in_size[1]
    H_out = math.floor((H_in+2*padding[0]-dilation[0]*(kernel_size[0]-1)-1)/stride[0]+1)
    W_out = math.floor((W_in+2*padding[1]-dilation[1]*(kernel_size[1]-1)-1)/stride[1]+1)
    return (H_out,W_out)

# CONVTRANSPOSE2D->https://pytorch.org/docs/stable/generated/torch.nn.ConvTranspose2d.html
def ConvTranspose2d_OutputShape(in_size,kernel_size,stride=(1,1),padding=(0,0),output_padding=(0,0),dilation=(1,1)):
    H_in = in_size[0]
    W_in = in_size[1]
    H_out = (H_in-1)*stride[0]-2*padding[0]+dilation[0]*(kernel_size[0]-1)+output_padding[0]+1
    W_out = (W_in-1)*stride[1]-2*padding[1]+dilation[1]*(kernel_size[1]-1)+output_padding[1]+1
    return (H_out,W_out)

# q(z|x, y)
in_size = (32,100)
out_size1 = Conv2d_OutputShape(in_size,(4,4),stride=(2,2),padding=(1,1))
out_size2 = Conv2d_OutputShape(out_size1,(4,4),stride=(2,2),padding=(1,1))
print(f"- q(z|x,y) : {in_size} ⇒ {out_size1} ⇒ {out_size2}")

# q(y|x)
out_size3 = Conv2d_OutputShape(in_size,(4,4),padding=(2,2))
out_size4 = MaxPool2d_OutputShape(out_size3,(2,2))
out_size5 = Conv2d_OutputShape(out_size4,(4,4),padding=(2,2))
out_size6 = MaxPool2d_OutputShape(out_size5,(2,2))
print(f"- q(y|x)   : {in_size} ⇒ {out_size3} ⇒ {out_size4} ⇒ {out_size5} ⇒ {out_size6}")

# p(x|z)
in_size_d = (in_size[0]//4, in_size[1]//4)
out_size7 = ConvTranspose2d_OutputShape(in_size_d,(4,4),stride=(2,2),padding=(1,1))
out_size8 = ConvTranspose2d_OutputShape(out_size7,(4,4),stride=(2,2),padding=(1,1))
print(f"- p(x|z)   : {in_size_d} ⇒ {out_size7} ⇒ {out_size8}")

- q(z|x,y) : (32, 100) ⇒ (16, 50) ⇒ (8, 25)
- q(y|x)   : (32, 100) ⇒ (33, 101) ⇒ (16, 50) ⇒ (17, 51) ⇒ (8, 25)
- p(x|z)   : (8, 25) ⇒ (16, 50) ⇒ (32, 100)


In [3]:
INPUT_HEIGHT = 32
INPUT_WIDTH = 100

# q(z|x, y)
class Qz_xy(nn.Module):
    def __init__(self, z_dim=2, y_dim=10):
        super(Qz_xy, self).__init__()

        self.z_dim = z_dim

        # encode
        self.conv_e = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=4, stride=2, padding=1),  # (32,100) ⇒ (16, 50)
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),  # (16, 50) ⇒ (8, 25)
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
        )
        self.fc1 = nn.Sequential(
            nn.Linear((INPUT_HEIGHT//4) * (INPUT_WIDTH//4)* 128,  40),
        )
        self.fc2 = nn.Sequential(
            nn.Linear((INPUT_HEIGHT//4) * (INPUT_WIDTH//4)* 128,  y_dim),
        )

        self.fc = nn.Sequential(
            nn.Linear(40+y_dim, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 2*self.z_dim),
        )

    def forward(self, x, y):
        x = self.conv_e(x)
        x = x.view(-1,(INPUT_HEIGHT//4) * (INPUT_WIDTH//4)* 128)
        x1 = self.fc1(x)
        x2 = self.fc2(x)
        x = torch.cat([x1, x2*y], dim=1)
        x = self.fc(x)
        mu = x[:, :self.z_dim]
        logvar = x[:, self.z_dim:]
        z = self.reparameterize(mu, logvar)
        self.mu = mu
        self.logvar = logvar
        return z

    def reparameterize(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = std.new(std.size()).normal_()
            return eps.mul(std).add_(mu)
        else:
            return mu


# q(y|x)
class Qy_x(nn.Module):
    def __init__(self, y_dim=10):
        super(Qy_x, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=4,padding=2),   # (32,100) ⇒ (33, 101)
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)) # (33, 101) ⇒ (16, 50)

        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=4, padding=2), # (16, 50) ⇒ (17, 51)
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2)) # (17, 51) ⇒ (8, 25)

        self.fc =  nn.Sequential(
            nn.Linear(-(-INPUT_HEIGHT//4) * -(-INPUT_WIDTH//4)* 128, 256),
            nn.Dropout(p=0.4),
            nn.ReLU(),
            nn.Linear(256, y_dim),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        c1 = self.conv1(x)
        c2 = self.conv2(c1)
        c2_flat = c2.view(c2.size(0), -1)
        out = self.fc(c2_flat)
        return out

# p(z|y)
class Pz_y(nn.Module):
    def __init__(self, z_dim=2, y_dim=10):
        super(Pz_y, self).__init__()

        self.z_dim = z_dim

        # encode
        self.fc = nn.Sequential(
            nn.Linear(y_dim, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 2*self.z_dim),
        )

    def forward(self, y):
        x = self.fc(y)
        mu = x[:, :self.z_dim]
        logvar = x[:, self.z_dim:]
        z = self.reparameterize(mu, logvar)
        self.mu = mu
        self.logvar = logvar
        return z

    def reparameterize(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = std.new(std.size()).normal_()
            return eps.mul(std).add_(mu)
        else:
            return mu

    def sample(self, y):
        x = self.fc(y)
        mu = x[:, :self.z_dim]
        logvar = x[:, self.z_dim:]
        std = logvar.mul(0.5).exp_()
        eps = std.new(std.size()).normal_()
        return eps.mul(std).add_(mu)

# p(x|z)
class Px_z(nn.Module):
    def __init__(self, z_dim=2):
        super(Px_z, self).__init__()

        self.z_dim = z_dim

        # decode
        self.fc = nn.Sequential(
            nn.Linear(self.z_dim, 40),
        )

        self.fc_d = nn.Sequential(
            nn.Linear(40, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024,(INPUT_HEIGHT//4) * (INPUT_WIDTH//4)* 128),
            nn.LeakyReLU(0.2)
        )
        self.conv_d = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1), # (8, 25) ⇒ (16, 50)
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),
            nn.ConvTranspose2d(64, 1, kernel_size=4, stride=2, padding=1), # (16, 50) ⇒ (32, 100) 元通り！
            nn.Sigmoid()
        )

    def forward(self, z):
        z = self.fc(z)
        h = self.fc_d(z)
        h = h.view(-1, 128, (INPUT_HEIGHT//4), (INPUT_WIDTH//4))
        #print(h.shape)
        return self.conv_d(h)

class MyDataset(torch.utils.data.Dataset):

    def __init__(self, data, label, datatime_idx, transform=None):
        self.transform = transform
        self.data = data
        self.data_num = len(data)
        self.label = label
        self.datatime_idx = datatime_idx

    def __len__(self):
        return self.data_num

    def __getitem__(self, idx):
        if self.transform:
            # print(self.data.shape)
            # print(self.data[idx].shape)
            out_data = self.transform(self.data[idx])
            out_label = int(self.label[idx])
            out_datatime_idx = int(self.datatime_idx[idx])
        else:
            out_data = self.data[idx]
            out_label =  self.label[idx]
            out_datatime_idx = self.datatime_idx[idx]

        return out_data, out_label, out_datatime_idx

## 全員のデータ対象に処理
ファイルリストに注意

In [4]:
# パラメータ
fs = 125
lpf_fp = 30
lpf_fs = 32
hpf_fp = 5
hpf_fs = 3
thinning_num = 4

dt = 1/fs
pre_sec = 0.5
post_sec = 0.3
pre_sample_num = int(pre_sec*fs)
post_sample_num = int(post_sec*fs)+1
N = pre_sample_num + post_sample_num # サンプル点数
time_array = np.arange(0,N)*dt # グラフ横軸（時間）

mother = pycwt.Morlet(6)
s0 = 2*dt # ウェーブレットの最小スケール。デフォルト値2dt。
dj = 1/12 # 離散スケールの間隔。デフォルト値1/12。
J =(math.log2(N * dt / s0))/dj # スケールの範囲s0からs0*2**(J*dj)までで、計(J+1)の尺度。
print(J)

# ファイルリスト読み込み
filtered_filelist = '../0_FileList/8_Raw.txt'
filelist_fp = codecs.open(filtered_filelist, 'r')

# ファイル読み込み
for idx, filename in enumerate(filelist_fp):
    split_fpdata = filename.rstrip('\r\n').split(',')
    fname = split_fpdata[0]
    fname = '../'+fname
    wave_fp = codecs.open(fname, 'r')

# ----- csvから2次元listへ
    times = []
    ch = []
    for i in range(7):
        ch.append([])
    for idx2, line in enumerate(wave_fp):
        split_data = line.rstrip('\r\n').split(',')
        times.append(idx2)
        for i in range(7):
            ch[i].append(int(float(split_data[i])))
    wave_fp.close()

    ################################################################################
    #                                フィルタ                                      #
    ################################################################################

    # PPGのフィルタは多分これで確定でよい
    ppg = sig_proc.lpf(ch[0], 500, fp=5, fs=10)[::thinning_num]  # PPG

    i_1st = ch[1]
    i_2nd = ch[2]
    q_1st = ch[3]
    q_2nd = ch[4]
    iq_diff_1st = [i-q for (i,q) in zip(i_1st, q_1st)]
    iq_diff_2nd = [i-q for (i,q) in zip(i_2nd, q_2nd)]

    # フィルタをかける
    i_1st = sig_proc.hpf(sig_proc.lpf(i_1st,500,fp=lpf_fp,fs=lpf_fs),500,fp=hpf_fp,fs=hpf_fs)[::thinning_num]
    i_2nd = sig_proc.hpf(sig_proc.lpf(i_2nd,500,fp=lpf_fp,fs=lpf_fs),500,fp=hpf_fp,fs=hpf_fs)[::thinning_num]
    q_1st = sig_proc.hpf(sig_proc.lpf(q_1st,500,fp=lpf_fp,fs=lpf_fs),500,fp=hpf_fp,fs=hpf_fs)[::thinning_num]
    q_2nd = sig_proc.hpf(sig_proc.lpf(q_2nd,500,fp=lpf_fp,fs=lpf_fs),500,fp=hpf_fp,fs=hpf_fs)[::thinning_num]
    iq_diff_1st = sig_proc.hpf(sig_proc.lpf(iq_diff_1st,500,fp=lpf_fp,fs=lpf_fs),500,fp=hpf_fp,fs=hpf_fs)[::thinning_num]
    iq_diff_2nd = sig_proc.hpf(sig_proc.lpf(iq_diff_2nd,500,fp=lpf_fp,fs=lpf_fs),500,fp=hpf_fp,fs=hpf_fs)[::thinning_num]

    # 呼吸ありデータ
    X_LIM_MIN = int(split_fpdata[1])/thinning_num
    X_LIM_MIN = int(X_LIM_MIN)
    X_LIM_MAX = X_LIM_MIN+30000/thinning_num #そこから60sは通常の呼吸
    X_LIM_MAX = int(X_LIM_MAX)

    chnp = [] 
    chnp.append(np.array(ppg)[X_LIM_MIN:X_LIM_MAX])
    chnp.append(np.array(i_1st)[X_LIM_MIN:X_LIM_MAX])
    chnp.append(np.array(i_2nd)[X_LIM_MIN:X_LIM_MAX])
    chnp.append(np.array(q_1st)[X_LIM_MIN:X_LIM_MAX])
    chnp.append(np.array(q_2nd)[X_LIM_MIN:X_LIM_MAX])
    chnp.append(np.array(iq_diff_1st)[X_LIM_MIN:X_LIM_MAX])
    chnp.append(np.array(iq_diff_2nd)[X_LIM_MIN:X_LIM_MAX])
    data_b = np.vstack([chnp[0], chnp[1], chnp[2], chnp[3], chnp[4], chnp[5], chnp[6]]) # 必要になればこれファイル出力すればOK
    

    # 呼吸なしデータ
    if int(split_fpdata[2]) == -1:
        X_LIM_MAX = -1
    else:
        X_LIM_MAX = int(split_fpdata[2])/thinning_num
        X_LIM_MAX = int(X_LIM_MAX)
    X_LIM_MIN = X_LIM_MAX - 5000/thinning_num
    X_LIM_MIN = int(X_LIM_MIN)

    chnp = []
    chnp.append(np.array(ppg)[X_LIM_MIN:X_LIM_MAX])
    chnp.append(np.array(i_1st)[X_LIM_MIN:X_LIM_MAX])
    chnp.append(np.array(i_2nd)[X_LIM_MIN:X_LIM_MAX])
    chnp.append(np.array(q_1st)[X_LIM_MIN:X_LIM_MAX])
    chnp.append(np.array(q_2nd)[X_LIM_MIN:X_LIM_MAX])
    chnp.append(np.array(iq_diff_1st)[X_LIM_MIN:X_LIM_MAX])
    chnp.append(np.array(iq_diff_2nd)[X_LIM_MIN:X_LIM_MAX])
    data_wob = np.vstack([chnp[0], chnp[1], chnp[2], chnp[3], chnp[4], chnp[5], chnp[6]])

    ################################################################################
    #                                　分割　                                      #
    ################################################################################
    # 一旦、呼吸なしデータだけを対象にして考える
    ppg = data_wob[0]
    len_data = data_wob.shape[1]

# ----- 時系列探索でPPGピーク検出＆分割
    peak_times, peak_vals = sig_proc.peak_search(sig_proc.min_max(ppg), fs)
    # 必要な部分だけ集めたデータdata_dist。あとから一拍ずつ分割。
    data_dist, npeaks, len_per_one, peak_times, peak_vals = peak_detect.data_distribution(data_wob, len_data, peak_times, peak_vals, pre_sample_num, post_sample_num)
    
    npeaks = data_dist.shape[0]

    fig = plt.figure(figsize=(18,10))
    
    # 時系列波形
    for peak_idx in range(npeaks):
        if peak_idx >= 9:
            continue

        # 呼吸なしデータ可視化
        doppler_data = data_dist[peak_idx][6]
        ppg_data = data_dist[peak_idx][0]

        ax1 = fig.add_subplot(3,3,peak_idx+1)
        ax1.plot(time_array, doppler_data, color='tab:blue', lw=5)
        ax2 = ax1.twinx()
        ax2.plot(time_array, ppg_data, color='tab:orange')
    #fig.tight_layout()
    fig.suptitle(fname, fontsize=30)
    plt.show()

    # CWT
    fig = plt.figure(figsize=(18,10))
    for peak_idx in range(npeaks):
        if peak_idx >= 9:
            continue

        doppler_data = data_dist[peak_idx][6]
        wave, _scales, freqs, _coi, _fft, _fftfreqs = pycwt.cwt(doppler_data, dt, dj, s0, J, mother)

        wave = wave[12:44,:]
        freqs = freqs[12:44]

        #wave = np.log(wave)
        #wave = sig_proc.min_max(wave) # 0-1正規化
        #wave = zscore(wave)
        
        ax = fig.add_subplot(3,3,peak_idx+1)
        ax.pcolormesh(time_array, freqs, np.abs(wave), vmin=0, shading='auto')
        ax.set_ylabel('Frequency [Hz]')
        ax.set_xlabel('Time [sec]')
        #ax.set_ylim([3, 50])

    #fig.tight_layout()
    fig.suptitle(fname, fontsize=30)
print(wave.shape)

67.7262742772967


In [ ]:
sample = np.array([[100,1,2],[3,4,5],[6,7,8],[9,10,11]])
freqs = np.array([60.50083182,57.10518106,53.90011352, 50.8749326])
t = np.arange(0,3) # グラフ横軸（時間）

fig = plt.figure()
ax = fig.add_subplot(111)
ax.pcolormesh(t, freqs, np.abs(sample), vmin=0, shading='auto')
ax.set_ylabel('Frequency [Hz]')
ax.set_xlabel('Time [sec]')
#ax.set_ylim([3, 50])

Error: Session cannot generate requests